# Python Algorithmic Trading Cookbook

## Chapter 9: Algorithmic Trading: Backtesting

This Jupyter Notebook is created using Python version 3.8.2

----

### Requirements

You can install the requirements for this Jupyter Notebook by executing the below cell

In [0]:
!pip install pyalgotrading

----

### Recipe 1: EMA-Regular-Order Strategy: Fetching the Strategy

In [1]:
from pyalgotrading.algobulls import AlgoBullsConnection

In [2]:
algobulls_connection = AlgoBullsConnection()

In [3]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [4]:
algobulls_connection.set_access_token('b4e0f923d085d48f4665a079d760d2bb94c33a94')

In [5]:
all_strategies = algobulls_connection.get_all_strategies()
all_strategies

strategyCode                 strategyName
0  49287246f9704bbcbad76ade9e2091d9   EMA Regular Order Strategy
1  4faf514fe096432b8e9f80f5951bd2ea  MACD Bracket Order Strategy

In [6]:
strategy_code1 = all_strategies.iloc[0]['strategyCode']
strategy_code1

'49287246f9704bbcbad76ade9e2091d9'

In [7]:
strategy_details1 = algobulls_connection.get_strategy_details(strategy_code1)
print(strategy_details1)

class StrategyEMARegularOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.timeperiod1 = self.strategy_parameters['timeperiod1']
        self.timeperiod2 = self.strategy_parameters['timeperiod2']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'EMA Regular Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        ema_x = talib.EMA(hist_data['close'], timeperiod=self.timeperiod1)
        ema_y = talib.EMA(hist_data['close'], timeperiod=self.timeperiod2)
        crossover_value = self.utils.crossover(ema_x, ema_y)
        return crossover_value

    def strategy_select_instruments_for_entry(self, candle, instruments_bucket):

        selected_instruments_bucket = 

### Recipe 2: EMA-Regular-Order Strategy: Backtesting the Strategy

In [8]:
from datetime import datetime as dt
from pyalgotrading.constants import *

In [9]:
instruments = algobulls_connection.search_instrument('SBIN')
instruments

[{'id': 7, 'value': 'NSE:SBIN'}]

In [10]:
instrument = instruments[0]['value']
instrument

'NSE:SBIN'

In [11]:
algobulls_connection.backtest(strategy_code=strategy_code1, 
                              start_timestamp=dt(year=2020, month=7, day=1, hour=9, minute=15), 
                              end_timestamp=dt(year=2020, month=7, day=7, hour=15, minute=30), 
                              instrument=instrument, 
                              lots=1,
                              strategy_parameters={
                                  'timeperiod1': 5,
                                  'timeperiod2': 12
                              }, 
                              candle_interval=CandleInterval.MINUTES_15)

Setting Strategy Config... Success.
Submitting BACKTESTING job... Success.


In [12]:
algobulls_connection.get_backtesting_job_status(strategy_code1)

{'data': 'STARTING'}

In [13]:
algobulls_connection.get_backtesting_job_status(strategy_code1)

{'data': 'STARTED'}

### There's more...

In [14]:
algobulls_connection.stop_backtesting_job(strategy_code1)

Stopping BACKTESTING job... Success.


In [15]:
algobulls_connection.get_backtesting_job_status(strategy_code1)

{'data': 'STOPPING'}

In [16]:
algobulls_connection.get_backtesting_job_status(strategy_code1)

{'data': 'STOPPED'}

<img src="images/diagrams/abc-job-status-state-diagram.png" width='50%'>

### Recipe 3: EMA-Regular-Order Strategy: Fetching Backtesting Logs in Realtime

In [17]:
logs = algobulls_connection.get_backtesting_logs(strategy_code1)
print(logs)

[2020-07-30 17:25:18] Logs not available yet. Please retry in sometime.


In [18]:
logs = algobulls_connection.get_backtesting_logs(strategy_code1)
print(logs)

[2020-07-30 11:56:29] Performing sanity checks on cfg strategy_parameters, setting up broker connection and required data structures...
[2020-07-30 11:56:29] ABBroker connection has been setup successfully.
[2020-07-30 11:56:29] Sanity checks on cfg successful.
[2020-07-30 11:56:29] Setting up broker connection...
[2020-07-30 11:56:29] Broker connection has been setup successfully.
[2020-07-30 11:56:29] (NSE_EQ) Funds available in client's ABVirtualBroker account is : Rs. '1000000000.00'
[2020-07-30 11:56:29] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.2.0 SECURE MODE)... 
########################################
[2020-07-30 11:56:29] Welcome ALGOBULLS VIRTUAL USER!
[2020-07-30 11:56:29] Reading strategy...
[2020-07-30 11:56:29] Entering Backtesting mode. Henceforth, all timestamps will be backtesting timestamps...
[BT] [2020-07-01 09:15:00] [INFO] [tls] STARTING ALGOBULLS CORE...
[BT] [2020-07-01 09:15:00] [INFO] [tls] 
            
  #####  #######    #

### Recipe 4: EMA-Regular-Order Strategy: Fetching Backtesting Reports - Profit-n-Loss Table

In [19]:
algobulls_connection.get_backtesting_report_pnl_table(strategy_code1, show_all_rows=True)

instrument      entry_timestamp entry_transaction_type entry_quantity  \
0  NSE_EQ:SBIN  2020-07-07 13:15:00                    BUY              1   
1  NSE_EQ:SBIN  2020-07-06 14:45:00                   SELL              1   
2  NSE_EQ:SBIN  2020-07-06 09:30:00                    BUY              1   
3  NSE_EQ:SBIN  2020-07-03 13:15:00                   SELL              1   
4  NSE_EQ:SBIN  2020-07-03 12:00:00                    BUY              1   
5  NSE_EQ:SBIN  2020-07-02 15:00:00                   SELL              1   
6  NSE_EQ:SBIN  2020-07-02 12:00:00                    BUY              1   
7  NSE_EQ:SBIN  2020-07-02 11:15:00                   SELL              1   
8  NSE_EQ:SBIN  2020-07-01 09:45:00                    BUY              1   

  entry_price       exit_timestamp exit_transaction_type exit_quantity  \
0      186.55  2020-07-07 15:30:00                  SELL             1   
1      187.85  2020-07-06 15:30:00                   BUY             1   
2      187.65  2020-07-06 14:45:00                  SELL             1   
3       184.6  2020-07-03 15:30:00                   BUY             1   
4      185.45  2020-07-03 13:15:00                  SELL             1   
5       185.6  2020-07-02 15:30:00                   BUY             1   
6      185.55  2020-07-02 15:00:00                  SELL             1   
7       184.4  2020-07-02 12:00:00                   BUY             1   
8      180.25  2020-07-01 15:30:00                  SELL             1   

  exit_price pnl_absolute pnl_percentage pnl_cumulative_absolute  \
0      188.0         1.45           0.78                    3.75   
1      187.9        -0.05          -0.03                     2.3   
2     187.85          0.2           0.11                    2.35   
3      184.6         -0.0           -0.0                    2.15   
4      184.6        -0.85          -0.46                    2.15   
5      185.7         -0.1          -0.05                     3.0   
6      185.6         0.05           0.03                     3.1   
7     185.55        -1.15          -0.62                    3.05   
8     184.45          4.2           2.33                     4.2   

  pnl_cumulative_percentage  
0                      2.09  
1                      1.31  
2                      1.34  
3                      1.23  
4                      1.23  
5                      1.69  
6                      1.74  
7                      1.71  
8                      2.33

### Recipe 5: EMA-Regular-Order Strategy: Fetching Backtesting Reports - Statistics Table

In [20]:
algobulls_connection.get_backtesting_report_statistics(strategy_code1)

highlight_type highlight_value
0                         Net PnL            3.75
1                       Net PnL %            2.09
2                    Max Drawdown            2.15
3                  Max Drawdown %            1.16
4                Number of Trades               9
5                  Number of Wins               5
6                Number of Looses               4
7           Number of Long Trades               5
8          Number of Short Trades               4
9                        Max Gain             4.2
10                       Min Gain           -1.15
11  Avg. Profit per winning trade            1.48
12   Avg. Profit per losing trade           -0.54

### Recipe 6: EMA-Regular-Order Strategy: Fetching Backtesting Reports - Order History

In [21]:
order_history = algobulls_connection.get_backtesting_report_order_history(strategy_code1)
print(order_history)



+-------------+---------------------+----------------------------------+------+
| INST        | TIME                | ID                               | TT   |
|-------------+---------------------+----------------------------------+------|
| NSE_EQ:SBIN | 2020-07-01 09:45:00 | 2333198611b744aeb287300d371c8eb5 | BUY  |
+-------------+---------------------+----------------------------------+------+
+----+---------------------+------------------------+-------+
|    | TIME                | STATE                  | MSG   |
|----+---------------------+------------------------+-------|
|  0 | 2020-07-01 09:45:00 | PUT ORDER REQ RECEIVED |       |
|  1 | 2020-07-01 09:45:00 | VALIDATION PENDING     |       |
|  2 | 2020-07-01 09:45:00 | OPEN PENDING           |       |
|  3 | 2020-07-01 09:45:00 | OPEN                   |       |
|  4 | 2020-07-01 09:45:00 | COMPLETE               |       |
+----+---------------------+------------------------+-------+

+-------------+---------------------+--

### Recipe 7: MACD-Bracket-Order Strategy: Fetching the Strategy

In [22]:
from pyalgotrading.algobulls import AlgoBullsConnection

In [23]:
algobulls_connection = AlgoBullsConnection()

In [24]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [25]:
algobulls_connection.set_access_token('b4e0f923d085d48f4665a079d760d2bb94c33a94')

In [26]:
all_strategies = algobulls_connection.get_all_strategies()
all_strategies

strategyCode                 strategyName
0  49287246f9704bbcbad76ade9e2091d9   EMA Regular Order Strategy
1  4faf514fe096432b8e9f80f5951bd2ea  MACD Bracket Order Strategy

In [27]:
strategy_code2 = all_strategies.iloc[1]['strategyCode']
strategy_code2

'4faf514fe096432b8e9f80f5951bd2ea'

In [28]:
strategy_details2 = algobulls_connection.get_strategy_details(strategy_code2)
print(strategy_details2)

class StrategyMACDBracketOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.fastMA_period = self.strategy_parameters['fastma_period']
        self.slowMA_period = self.strategy_parameters['slowma_period']
        self.signal_period = self.strategy_parameters['signal_period']
        self.stoploss = self.strategy_parameters['stoploss_trigger']
        self.target = self.strategy_parameters['target_trigger']
        self.trailing_stoploss = self.strategy_parameters['trailing_stoploss_trigger']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'MACD Bracket Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        macdline, macdsignal, _ = talib.MACD(hist_data['close

### Recipe 8: MACD-Bracket-Order Strategy: Backtesting the Strategy

In [29]:
from datetime import datetime as dt
from pyalgotrading.constants import *

In [30]:
instrument = algobulls_connection.search_instrument('TATASTEEL')
instrument

[{'id': 1, 'value': 'NSE:TATASTEEL'}]

In [31]:
instrument = instrument[0]['value']
instrument

'NSE:TATASTEEL'

In [32]:
algobulls_connection.backtest(strategy_code=strategy_code2, 
                              start_timestamp=dt(year=2020, month=7, day=1, hour=9, minute=15),
                              end_timestamp=dt(year=2020, month=7, day=7, hour=15, minute=30),
                              instrument=instrument,
                              lots=1,
                              strategy_parameters={
                                  'fastma_period': 26, 
                                  'slowma_period': 6, 
                                  'signal_period': 9,
                                  'target_trigger': 0.01, 
                                  'stoploss_trigger': 0.01, 
                                  'trailing_stoploss_trigger': 1
                              },
                              candle_interval=CandleInterval.MINUTES_15)

Setting Strategy Config... Success.
Submitting BACKTESTING job... Success.


In [33]:
algobulls_connection.get_backtesting_job_status(strategy_code2)

{'data': 'STARTING'}

In [34]:
algobulls_connection.get_backtesting_job_status(strategy_code2)

{'data': 'STARTED'}

### There's more...

In [35]:
algobulls_connection.stop_backtesting_job(strategy_code2)

Stopping BACKTESTING job... Success.


In [36]:
algobulls_connection.get_backtesting_job_status(strategy_code2)

{'data': 'STOPPING'}

In [37]:
algobulls_connection.get_backtesting_job_status(strategy_code2)

{'data': 'STOPPED'}

<img src="images/diagrams/abc-job-status-state-diagram.png" width='50%'>

### Recipe 9: MACD-Bracket-Order Strategy: Fetching Backtesting Logs in Realtime

In [38]:
logs = algobulls_connection.get_backtesting_logs(strategy_code2)
print(logs)

[2020-07-30 17:27:25] Logs not available yet. Please retry in sometime.


In [39]:
logs = algobulls_connection.get_backtesting_logs(strategy_code2)
print(logs)

[2020-07-30 12:00:58] Performing sanity checks on cfg strategy_parameters, setting up broker connection and required data structures...
[2020-07-30 12:00:58] ABBroker connection has been setup successfully.
[2020-07-30 12:00:58] Sanity checks on cfg successful.
[2020-07-30 12:00:58] Setting up broker connection...
[2020-07-30 12:00:58] Broker connection has been setup successfully.
[2020-07-30 12:00:58] (NSE_EQ) Funds available in client's ABVirtualBroker account is : Rs. '1000000000.00'
[2020-07-30 12:00:58] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.2.0 SECURE MODE)... 
########################################
[2020-07-30 12:00:58] Welcome ALGOBULLS VIRTUAL USER!
[2020-07-30 12:00:58] Reading strategy...
[2020-07-30 12:00:58] Entering Backtesting mode. Henceforth, all timestamps will be backtesting timestamps...
[BT] [2020-07-01 09:15:00] [INFO] [tls] STARTING ALGOBULLS CORE...
[BT] [2020-07-01 09:15:00] [INFO] [tls] 
            
  #####  #######    #

### Recipe 10: MACD-Bracket-Order Strategy: Fetching Backtesting Reports - Profit-n-Loss Table

In [40]:
algobulls_connection.get_backtesting_report_pnl_table(strategy_code2)

instrument      entry_timestamp entry_transaction_type  \
0  NSE_EQ:TATASTEEL  2020-07-07 14:45:00                   SELL   
1  NSE_EQ:TATASTEEL  2020-07-07 13:15:00                    BUY   
2  NSE_EQ:TATASTEEL  2020-07-06 13:30:00                   SELL   
3  NSE_EQ:TATASTEEL  2020-07-06 09:45:00                    BUY   
4  NSE_EQ:TATASTEEL  2020-07-03 12:45:00                   SELL   
5  NSE_EQ:TATASTEEL  2020-07-03 12:30:00                    BUY   
6  NSE_EQ:TATASTEEL  2020-07-03 09:45:00                   SELL   
7  NSE_EQ:TATASTEEL  2020-07-01 12:30:00                    BUY   

  entry_quantity entry_price       exit_timestamp exit_transaction_type  \
0              1       329.3  2020-07-07 15:30:00                   BUY   
1              1       332.6  2020-07-07 13:30:00                  SELL   
2              1      339.65  2020-07-06 15:30:00                   BUY   
3              1      332.35  2020-07-06 10:00:00                  SELL   
4              1       333.5  2020-07-03 13:00:00                   BUY   
5              1      334.45  2020-07-03 12:45:00                  SELL   
6              1       330.8  2020-07-03 10:00:00                   BUY   
7              1       322.6  2020-07-01 15:30:00                  SELL   

  exit_quantity exit_price pnl_absolute pnl_percentage  \
0             1      331.0         -1.7          -0.52   
1             1     329.25        -3.35          -1.01   
2             1     338.85          0.8           0.24   
3             1     335.65          3.3           0.99   
4             1     330.15         3.35            1.0   
5             1      333.5        -0.95          -0.28   
6             1      334.1         -3.3           -1.0   
7             1      324.1          1.5           0.46   

  pnl_cumulative_absolute pnl_cumulative_percentage  
0                   -0.35                     -0.12  
1                    1.35                       0.4  
2                     4.7                      1.41  
3                     3.9                      1.17  
4                     0.6                      0.18  
5                   -2.75                     -0.82  
6                    -1.8                     -0.54  
7                     1.5                      0.46

### Recipe 11: MACD-Bracket-Order Strategy: Fetching Backtesting Reports - Statistics Table

In [41]:
algobulls_connection.get_backtesting_report_statistics(strategy_code2)

highlight_type highlight_value
0                         Net PnL           -0.35
1                       Net PnL %           -0.12
2                    Max Drawdown           -2.75
3                  Max Drawdown %           -0.82
4                Number of Trades               8
5                  Number of Wins               4
6                Number of Looses               4
7           Number of Long Trades               4
8          Number of Short Trades               4
9                        Max Gain            3.35
10                       Min Gain           -3.35
11  Avg. Profit per winning trade            2.24
12   Avg. Profit per losing trade           -2.32

### Recipe 12: MACD-Bracket-Order Strategy: Fetching Backtesting Reports - Order History

In [42]:
order_history = algobulls_connection.get_backtesting_report_order_history(strategy_code2)
print(order_history)



+------------------+---------------------+----------------------------------+------+
| INST             | TIME                | ID                               | TT   |
|------------------+---------------------+----------------------------------+------|
| NSE_EQ:TATASTEEL | 2020-07-01 12:30:00 | 1cbefcf395c344c88a228a1b01c32ef6 | BUY  |
+------------------+---------------------+----------------------------------+------+
+----+---------------------+------------------------+-------+
|    | TIME                | STATE                  | MSG   |
|----+---------------------+------------------------+-------|
|  0 | 2020-07-01 12:30:00 | PUT ORDER REQ RECEIVED |       |
|  1 | 2020-07-01 12:30:00 | VALIDATION PENDING     |       |
|  2 | 2020-07-01 12:30:00 | OPEN PENDING           |       |
|  3 | 2020-07-01 12:30:00 | OPEN                   |       |
|  4 | 2020-07-01 12:30:00 | COMPLETE               |       |
+----+---------------------+------------------------+-------+

+-------------